In [1]:
import os
import pandas as pd

In [2]:
from configparser import ConfigParser
cfg = ConfigParser()
cfg.read("config.cfg")

['config.cfg']

In [ ]:
from tqdm._tqdm_notebook import tqdm_notebook as tqdm # we manually import the notebook submodule as the normal one struggles with jupyterlab
tqdm.pandas() # this enables us to use progress_apply instead of apply

In [ ]:
df = pd.read_pickle(os.path.join(cfg.get("directory", "exchange"), "[wikidata]002_extract_actions.p"))

In [ ]:
df.head()

### Load mapping if we have one

In [ ]:
label_dict = {}
mapping_file = os.path.join(cfg.get("directory", "dataset"), cfg.get("dataset", "wikidata_aux"), cfg.get("dataset", "wikidata_full_labels"))
if os.path.isfile(mapping_file): # load it manually if we have a mapping already
    df_labels_load = pd.read_csv(mapping_file)
    for i, r in df_labels_load.iterrows():
        label_dict[r['action_base']] = r['label']
# label_dict

In [ ]:
action_counts = df['action_base'].value_counts()

### Manual mapping for undefined actions

Wikidata API reference: https://www.mediawiki.org/wiki/Wikibase/API/en#wblinktitles

In [ ]:
for a,c in action_counts.iteritems():
    if a not in label_dict:
        print(a, c)
        label = input()
        label_dict[a] = label

In [ ]:
df_label = pd.DataFrame({"action_base": list(label_dict.keys()), "label": list(label_dict.values())})

In [ ]:
# store labels
df_label.to_csv(mapping_file, index=False)

### Add label to revision

In [ ]:
len(df)

In [ ]:
df = df.merge(right=df_label, how="left", on="action_base")

In [ ]:
df.head()

In [ ]:
labels_to_remove = df['label'].value_counts()
labels_to_remove = list(labels_to_remove.loc[labels_to_remove <= cfg.getint("preprocessing", "min_label_occurrency")].index)
labels_to_remove

In [ ]:
print("Num revisions before removal: {n}".format(n=len(df)))
df = df.loc[~df['label'].isin(labels_to_remove)].reset_index(drop=True)
print("Num revisions after removal: {n}".format(n=len(df)))

In [ ]:
print("Num unique labels: {n}".format(n=len(df['label'].unique())))

In [ ]:
df['label'].value_counts()

### Store data

In [ ]:
df.to_pickle(os.path.join(cfg.get("directory", "exchange"), "[wikidata]003_assign_labels.p"))